# Get Retriver

In [1]:
import sys

# Add the directory to sys.path
sys.path.append('/home/qianyucazelles/internal_knowledge_chatbot/src/utils')
from doc_to_vertex_search import *

In [2]:
parameters = get_me_parameters('../vector_store_me_parameters/ecg_all_me.json')

In [3]:
embeddings = get_embeddings()

In [4]:
db = get_vector_store(parameters, embeddings)

In [5]:
from google.cloud.aiplatform.matching_engine.matching_engine_index_endpoint import (
    Namespace,
    NumericNamespace,
)
filters = [Namespace(name="department", allow_tokens=["dpo"]),
           Namespace(name="loader_method", allow_tokens=["unstructured"])
          ]


numeric_filters = [NumericNamespace(name="chunk_size", value_float=2000.0, op="EQUAL")]


In [14]:
retriever = db.as_retriever()

In [15]:
retriever.search_kwargs = {"filter": filters, "numeric_filter": numeric_filters}

In [19]:
q = "Je n'aurai pas le temps de traiter la demande de droit à l'oubli d'un utilisateur dans le mois règlementaire imposé par la CNIL. Ai-je un recours ?"

In [10]:
q2 = "J'ai des difficultés à comprendre les différentes obligations que j'ai vis-à-vis des sous-traitants qui travaillent pour moi. Pourrais-tu me faire un résumé ?"

In [12]:
retrieved_documents = retriever.invoke(q)

NameError: name 'retriever' is not defined

In [16]:
retrieved_documents = retriever.invoke(q2)

## local llm

In [42]:
# https://huggingface.co/blog/not-lain/rag-chatbot-using-llama3

In [1]:
from transformers import AutoTokenizer, AutoModelForCausalLM, BitsAndBytesConfig
import torch

model_id = "meta-llama/Meta-Llama-3-8B-Instruct"

# use quantization to lower GPU usage
bnb_config = BitsAndBytesConfig(
    load_in_4bit=True, bnb_4bit_use_double_quant=True, bnb_4bit_quant_type="nf4", bnb_4bit_compute_dtype=torch.bfloat16
)

tokenizer = AutoTokenizer.from_pretrained(model_id)
model = AutoModelForCausalLM.from_pretrained(
    model_id,
    torch_dtype=torch.bfloat16,
    device_map="auto",
    quantization_config=bnb_config
)
terminators = [
    tokenizer.eos_token_id,
    tokenizer.convert_tokens_to_ids("<|eot_id|>")
]

/home/qianyucazelles/internal_knowledge_chatbot/venv/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.
Loading checkpoint shards:  75%|███████▌  | 3/4 [00:08<00:02,  2.68s/it]Exception ignored in: <bound method IPythonKernel._clean_thread_parent_frames of <ipykernel.ipkernel.IPythonKernel object at 0x7fe592f79630>>
Traceback (most recent call last):
  File "/home/qianyucazelles/internal_knowledge_chatbot/venv/lib/python3.10/site-packages/ipykernel/ipkernel.py", line 775, in _clean_thread_parent_frames
    def _clean_thread_parent_frames(
KeyboardInterrupt: 
Loading checkpoint shards: 100%|██████████| 4/4 [00:08<00:00,  2.15s/it]


In [2]:
model.device

device(type='cuda', index=0)

In [7]:
SYS_PROMPT = """You are an assistant for answering questions.
You are given the extracted parts of several documents rank by relevance with document name and a question. Please pick the most relevant document related to the question then 
provide a conversational answer in the same language as the question. Always cite the referred document in your response.
If you don't know the answer, just say "I do not know." Don't make up an answer."""

In [8]:
def format_prompt(prompt,retrieved_documents):
  """using the retrieved documents we will prompt the model to generate our responses"""
  formatted_prompt = f"Question:{prompt}\nContext: \n"
  separated_line = "-"*50+"\n"
  for idx,doc in enumerate(retrieved_documents) :
    formatted_prompt+= f"{separated_line} Ranked Document: {idx+1} \nName: {doc.metadata['document_name']}\nContent: {doc.page_content} \n"
  return formatted_prompt

In [17]:
formatted_prompt = format_prompt(q2,retrieved_documents)

In [18]:
messages = [{"role":"system","content":SYS_PROMPT},{"role":"user","content":formatted_prompt}]

In [19]:
from transformers import TextIteratorStreamer

from threading import Thread

In [24]:
llm_top_p = 0.75
llm_top_k = 20
llm_temperature = 0.1

In [20]:
# set streamer
streamer = TextIteratorStreamer(tokenizer, skip_prompt=True, skip_special_tokens=True)


# apply chat template
messages_tmpl = tokenizer.apply_chat_template(messages,
                       tokenize=False,
                       add_generation_prompt=True
                      )

# tokenize msgs
inputs = tokenizer([messages_tmpl], return_tensors="pt").to('cuda')



Setting `pad_token_id` to `eos_token_id`:128009 for open-end generation.
2024-05-20 18:18:52.495934: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.


In [25]:
# Run the generation in a separate thread, so that we can fetch the generated text in a non-blocking way.
generation_kwargs = dict(inputs, 
                         streamer=streamer, 
                         max_new_tokens=1500,
                         eos_token_id=terminators,
                         top_k= llm_top_k,
                         top_p= llm_top_p,
                        do_sample=True,
                        temperature=llm_temperature,
                         )
# chat
thread = Thread(target=model.generate,
                kwargs=generation_kwargs)
thread.start() 

generated_text = ''
for _, new_text in enumerate(streamer):
    generated_text += new_text


Setting `pad_token_id` to `eos_token_id`:128009 for open-end generation.


In [26]:
print(generated_text)

Bonjour! Je suis là pour vous aider.

En ce qui concerne vos difficultés à comprendre les obligations que vous avez envers les sous-traitants qui travaillent pour vous, je vais vous donner un résumé basé sur le document le plus pertinent, qui est le CHAPITRE IV - Responsable du traitement et sous-traitant du CNIL.

Selon ce document, en tant que responsable du traitement, vous devez vous assurer que les sous-traitants que vous embauchent présentent des garanties suffisantes pour mettre en œuvre des mesures techniques et organisationnelles appropriées pour protéger les données personnelles. Vous devez également vous assurer que les sous-traitants ont des obligations claires et précises pour protéger les données personnelles, notamment :

* Informer immédiatement le responsable du traitement si une instruction constitue une violation du règlement ou d'autres dispositions du droit de l'Union ou du droit des États membres relatives à la protection des données.
* Présenter des garanties suf

In [22]:
print(generated_text)

Bonjour! Je suis là pour vous aider.

Selon le document le plus pertinent, le CHAPITRE IV - Responsable du traitement et sous-traitant _ CNIL.pdf, il est important de noter que les sous-traitants doivent présenter des garanties suffisantes quant à la mise en œuvre de mesures techniques et organisationnelles appropriées pour protéger les données des personnes concernées.

En ce qui concerne vos obligations envers les sous-traitants, vous devez notamment :

* Informer immédiatement le sous-traitant si, selon vous, une instruction constitue une violation du règlement ou d'autres dispositions du droit de l'Union ou du droit des États membres relatives à la protection des données.
* Imposer des obligations de protection des données à tout autre sous-traitant recruté pour mener des activités de traitement spécifiques pour votre compte.
* Veiller à ce que les sous-traitants qui traitent des données à votre place soient soumis à des garanties suffisantes quant à la mise en œuvre de mesures tec